In [1]:
from helper.load_scalars import read_csv_files
from helper.check_count import check_count

import re
import statistics

In [2]:
directory_path = "./runs"
exp_data = read_csv_files(directory_path)

test_accuracy$s168740-cl1-cf128-dm-bn0.csv
test_accuracy$s168740-cl1-cf128-dm-bn1.csv
test_accuracy$s168740-cl1-cf128-dn-bn0.csv
test_accuracy$s168740-cl1-cf128-dn-bn1.csv
test_accuracy$s168740-cl1-cf128-ds-bn0.csv
test_accuracy$s168740-cl1-cf128-ds-bn1.csv
test_accuracy$s168740-cl1-cf16-dm-bn0.csv
test_accuracy$s168740-cl1-cf16-dm-bn1.csv
test_accuracy$s168740-cl1-cf16-dn-bn0.csv
test_accuracy$s168740-cl1-cf16-dn-bn1.csv
test_accuracy$s168740-cl1-cf16-ds-bn0.csv
test_accuracy$s168740-cl1-cf16-ds-bn1.csv
test_accuracy$s168740-cl1-cf32-dm-bn0.csv
test_accuracy$s168740-cl1-cf32-dm-bn1.csv
test_accuracy$s168740-cl1-cf32-dn-bn0.csv
test_accuracy$s168740-cl1-cf32-dn-bn1.csv
test_accuracy$s168740-cl1-cf32-ds-bn0.csv
test_accuracy$s168740-cl1-cf32-ds-bn1.csv
test_accuracy$s168740-cl1-cf64-dm-bn0.csv
test_accuracy$s168740-cl1-cf64-dm-bn1.csv
test_accuracy$s168740-cl1-cf64-dn-bn0.csv
test_accuracy$s168740-cl1-cf64-dn-bn1.csv
test_accuracy$s168740-cl1-cf64-ds-bn0.csv
test_accuracy$s168740-cl1-cf

In [3]:
count_dict = check_count(directory_path)

got 5 seeds: ['s168740', 's241223', 's314159', 's424242', 's856263']
# s168740: 9 metrics: ['test_accuracy', 'test_f1', 'test_loss', 'train_accuracy', 'train_f1', 'train_loss', 'validation_accuracy', 'validation_f1', 'validation_loss']
# s241223: 9 metrics: ['test_accuracy', 'test_f1', 'test_loss', 'train_accuracy', 'train_f1', 'train_loss', 'validation_accuracy', 'validation_f1', 'validation_loss']
# s314159: 9 metrics: ['test_accuracy', 'test_f1', 'test_loss', 'train_accuracy', 'train_f1', 'train_loss', 'validation_accuracy', 'validation_f1', 'validation_loss']
# s424242: 9 metrics: ['test_accuracy', 'test_f1', 'test_loss', 'train_accuracy', 'train_f1', 'train_loss', 'validation_accuracy', 'validation_f1', 'validation_loss']
# s856263: 9 metrics: ['test_accuracy', 'test_f1', 'test_loss', 'train_accuracy', 'train_f1', 'train_loss', 'validation_accuracy', 'validation_f1', 'validation_loss']
---
# s168740 # test_accuracy: 120 models
# s168740 # test_f1: 120 models
# s168740 # test_loss:

In [4]:
def model_name_sort(string):
    # Extracting cl and cf numbers using regular expressions
    match = re.match(r'cl(\d+)-cf(\d+)', string)
    if match:
        cl_number = int(match.group(1))
        cf_number = int(match.group(2))
        return (cl_number, cf_number)
    else:
        # If the string doesn't match the expected pattern, place it at the end
        return (float('inf'), float('inf'))

In [5]:
def get_averages(metric_data, metric, filter='-dn-bn0', printStats=True):
    labels = []
    data = []
    for model, results in metric_data[metric].items():
        if not model.endswith(filter): continue
        # print(results)
        model_data=[]
        labels.append(model)
        for result in results:
            model_data.append(result['y'][0])
        data.append(model_data)

    combined_data = list(zip(labels, data))
    sorted_combined_data = sorted(combined_data, key=lambda x: model_name_sort(x[0]))
    sorted_labels, sorted_data = zip(*sorted_combined_data)
    
    if printStats:
        print(metric)
        for idx, label in enumerate(sorted_labels):
            print(f"{label: <16} - avg: {statistics.fmean(sorted_data[idx]):.3f} - {sorted_data[idx]}")
    return sorted_labels, sorted_data


In [6]:
def get_generalization_gap(metric_data, filter='-dn-bn0', printStats=True):
    labels = []
    data = []
    for model in metric_data['validation_loss']:
        if not model.endswith(filter): continue
        model_ggs = []
        for idx, seed in enumerate(metric_data['validation_loss'][model]):
            val_loss = metric_data['validation_loss'][model][idx]['y'][-1]
            # train_loss_seed_idx = metric_data['train_loss'][model]
            # print(train_loss_seed_idx)
            train_loss = metric_data['train_loss'][model][idx]['y'][-1]
            
            model_ggs.append(val_loss-train_loss)
        labels.append(model)
        data.append(model_ggs)
    
    combined_data = list(zip(labels, data))
    sorted_combined_data = sorted(combined_data, key=lambda x: model_name_sort(x[0]))
    sorted_labels, sorted_data = zip(*sorted_combined_data)
    if printStats:
        for idx, label in enumerate(sorted_labels):
            print(f"{label: <16} - avg delta: {statistics.fmean(sorted_data[idx]):.3f} - {sorted_data[idx]}")
    return sorted_labels, sorted_data

In [7]:
data = get_averages(exp_data, 'test_accuracy', filter='-dn-bn0')

test_accuracy
cl1-cf16-dn-bn0  - avg: 0.785 - [0.787804901599884, 0.795121967792511, 0.7585365772247314, 0.790243923664093, 0.792682945728302]
cl1-cf32-dn-bn0  - avg: 0.802 - [0.8024390339851379, 0.8048780560493469, 0.800000011920929, 0.8073170781135559, 0.79756098985672]
cl1-cf64-dn-bn0  - avg: 0.791 - [0.8024390339851379, 0.795121967792511, 0.8048780560493469, 0.7585365772247314, 0.792682945728302]
cl1-cf128-dn-bn0 - avg: 0.803 - [0.8146341443061829, 0.792682945728302, 0.795121967792511, 0.8048780560493469, 0.8097561001777649]
cl2-cf16-dn-bn0  - avg: 0.795 - [0.8073170781135559, 0.792682945728302, 0.7829268574714661, 0.8024390339851379, 0.790243923664093]
cl2-cf32-dn-bn0  - avg: 0.818 - [0.8268292546272278, 0.8121951222419739, 0.8097561001777649, 0.8195121884346008, 0.8219512104988098]
cl2-cf64-dn-bn0  - avg: 0.814 - [0.8390243649482727, 0.792682945728302, 0.800000011920929, 0.8170731663703918, 0.8195121884346008]
cl2-cf128-dn-bn0 - avg: 0.812 - [0.8048780560493469, 0.809756100177764

In [8]:
data = get_averages(exp_data, 'test_f1', filter='-dn-bn0')

test_f1
cl1-cf16-dn-bn0  - avg: 0.828 - [0.8323699235916138, 0.833992063999176, 0.8047336935997009, 0.83203125, 0.8355900049209595]
cl1-cf32-dn-bn0  - avg: 0.839 - [0.8408644199371338, 0.8393574357032776, 0.8366533517837524, 0.8423153758049011, 0.8356435298919678]
cl1-cf64-dn-bn0  - avg: 0.828 - [0.8389661908149719, 0.8306451439857483, 0.8418972492218018, 0.7999999523162842, 0.8303393721580505]
cl1-cf128-dn-bn0 - avg: 0.839 - [0.8480000495910645, 0.8261758685112, 0.8352941274642944, 0.844357967376709, 0.8427419662475586]
cl2-cf16-dn-bn0  - avg: 0.832 - [0.8435643315315247, 0.8275861740112305, 0.8244576454162598, 0.8383233547210693, 0.827309250831604]
cl2-cf32-dn-bn0  - avg: 0.848 - [0.853608250617981, 0.8392484188079834, 0.8427419662475586, 0.8502024412155151, 0.8554455637931824]
cl2-cf64-dn-bn0  - avg: 0.843 - [0.8695651888847351, 0.8254620432853699, 0.8305785059928894, 0.8459959030151367, 0.8451882600784302]
cl2-cf128-dn-bn0 - avg: 0.846 - [0.8425197005271912, 0.84765625, 0.843137204

In [9]:
data = get_generalization_gap(exp_data, filter='-dn-bn1')

cl1-cf16-dn-bn1  - avg delta: 0.026 - [0.011713743209838867, 0.027713418006896973, 0.032885611057281494, 0.037366896867752075, 0.01872551441192627]
cl1-cf32-dn-bn1  - avg delta: 0.023 - [0.018937915563583374, 0.03234592080116272, 0.025905489921569824, 0.017457187175750732, 0.022110074758529663]
cl1-cf64-dn-bn1  - avg delta: 0.032 - [0.03795459866523743, 0.016178131103515625, 0.03424948453903198, 0.027027159929275513, 0.04500865936279297]
cl1-cf128-dn-bn1 - avg delta: 0.050 - [0.07607215642929077, 0.06843692064285278, 0.04157164692878723, 0.022293776273727417, 0.040102481842041016]
cl2-cf16-dn-bn1  - avg delta: 0.109 - [0.20302459597587585, 0.09875687956809998, 0.0956203043460846, 0.0709543526172638, 0.07891446352005005]
cl2-cf32-dn-bn1  - avg delta: 0.194 - [0.3667784631252289, 0.2717168927192688, 0.1079791784286499, 0.117593914270401, 0.10601919889450073]
cl2-cf64-dn-bn1  - avg delta: 0.215 - [0.18210506439208984, 0.28707101941108704, 0.23260869085788727, 0.1555657535791397, 0.2160113

In [10]:
labels, intial = get_generalization_gap(exp_data, filter='-dn-bn0', printStats=False)
_, bn = get_generalization_gap(exp_data, filter='-dn-bn1', printStats=False)
_, ds = get_generalization_gap(exp_data, filter='-ds-bn0', printStats=False)
_, dm = get_generalization_gap(exp_data, filter='-dm-bn0', printStats=False)
_, dsbn = get_generalization_gap(exp_data, filter='-ds-bn1', printStats=False)
_, dmbn = get_generalization_gap(exp_data, filter='-dm-bn1', printStats=False)

print(f"{'model':^10} | {'intial':^8} | {'bn':^8} | {'ds':^8} | {'dm':^8} | {'dsbn':^8} | {'dmbn':^8}")
for i in range(20):
    intial_avg = statistics.fmean(intial[i])
    print(f"{labels[i].replace('-dn-bn0',''): <10} | {intial_avg:.3f}    | {statistics.fmean(bn[i])-intial_avg:+.3f}   | {statistics.fmean(ds[i])-intial_avg:+.3f}   | {statistics.fmean(dm[i])-intial_avg:+.3f}   | {statistics.fmean(dsbn[i])-intial_avg:+.3f}   | {statistics.fmean(dmbn[i])-intial_avg:+.3f}")

  model    |  intial  |    bn    |    ds    |    dm    |   dsbn   |   dmbn  
cl1-cf16   | 0.000    | +0.025   | -0.023   | -0.020   | -0.021   | -0.020
cl1-cf32   | 0.002    | +0.021   | -0.033   | -0.026   | -0.021   | -0.007
cl1-cf64   | 0.003    | +0.029   | -0.020   | -0.023   | -0.009   | -0.021
cl1-cf128  | 0.006    | +0.044   | -0.032   | -0.025   | -0.007   | -0.001
cl2-cf16   | 0.011    | +0.099   | -0.031   | -0.013   | -0.012   | +0.001
cl2-cf32   | 0.019    | +0.175   | -0.043   | -0.039   | +0.003   | -0.012
cl2-cf64   | 0.026    | +0.189   | -0.044   | -0.047   | +0.069   | +0.003
cl2-cf128  | 0.045    | +0.233   | -0.067   | -0.075   | +0.018   | -0.034
cl3-cf16   | 0.028    | +0.151   | -0.043   | -0.013   | +0.017   | -0.019
cl3-cf32   | 0.057    | +0.262   | -0.074   | -0.055   | +0.091   | -0.002
cl3-cf64   | 0.100    | +0.285   | -0.110   | -0.117   | +0.093   | +0.048
cl3-cf128  | 0.138    | +0.335   | -0.114   | -0.145   | +0.014   | -0.040
cl4-cf16   | 0.059    |

In [11]:
metric = 'test_accuracy'
labels, intial = get_averages(exp_data, metric, filter='-dn-bn0', printStats=False)
_, bn = get_averages(exp_data, metric, filter='-dn-bn1', printStats=False)
_, ds = get_averages(exp_data, metric, filter='-ds-bn0', printStats=False)
_, dm = get_averages(exp_data, metric, filter='-dm-bn0', printStats=False)
_, dsbn = get_averages(exp_data, metric, filter='-ds-bn1', printStats=False)
_, dmbn = get_averages(exp_data, metric, filter='-dm-bn1', printStats=False)

print(f"{'model':^10} | {'intial':^8} | {'bn':^8} | {'ds':^8} | {'dm':^8} | {'dsbn':^8} | {'dmbn':^8}")
for i in range(20):
    intial_avg = statistics.fmean(intial[i])
    print(f"{labels[i].replace('-dn-bn0',''): <10} | {intial_avg:.3f}    | {statistics.fmean(bn[i])-intial_avg:+.3f}   | {statistics.fmean(ds[i])-intial_avg:+.3f}   | {statistics.fmean(dm[i])-intial_avg:+.3f}   | {statistics.fmean(dsbn[i])-intial_avg:+.3f}   | {statistics.fmean(dmbn[i])-intial_avg:+.3f}")

  model    |  intial  |    bn    |    ds    |    dm    |   dsbn   |   dmbn  
cl1-cf16   | 0.785    | -0.009   | -0.027   | -0.032   | -0.014   | -0.017
cl1-cf32   | 0.802    | -0.020   | -0.035   | -0.033   | -0.027   | -0.034
cl1-cf64   | 0.791    | -0.008   | -0.022   | -0.018   | -0.026   | -0.023
cl1-cf128  | 0.803    | -0.027   | -0.026   | -0.031   | -0.034   | -0.027
cl2-cf16   | 0.795    | +0.009   | -0.004   | -0.006   | +0.008   | -0.027
cl2-cf32   | 0.818    | -0.002   | -0.018   | -0.019   | -0.016   | -0.027
cl2-cf64   | 0.814    | -0.007   | -0.025   | -0.011   | -0.046   | -0.035
cl2-cf128  | 0.812    | -0.004   | -0.008   | -0.010   | -0.018   | -0.009
cl3-cf16   | 0.829    | +0.013   | -0.011   | -0.051   | +0.025   | -0.046
cl3-cf32   | 0.843    | +0.006   | -0.022   | -0.051   | -0.015   | -0.034
cl3-cf64   | 0.845    | +0.009   | -0.020   | -0.034   | -0.037   | -0.060
cl3-cf128  | 0.850    | -0.017   | -0.023   | -0.032   | -0.012   | -0.045
cl4-cf16   | 0.846    |

In [12]:
metric = 'test_f1'
labels, intial = get_averages(exp_data, metric, filter='-dn-bn0', printStats=False)
_, bn = get_averages(exp_data, metric, filter='-dn-bn1', printStats=False)
_, ds = get_averages(exp_data, metric, filter='-ds-bn0', printStats=False)
_, dm = get_averages(exp_data, metric, filter='-dm-bn0', printStats=False)
_, dsbn = get_averages(exp_data, metric, filter='-ds-bn1', printStats=False)
_, dmbn = get_averages(exp_data, metric, filter='-dm-bn1', printStats=False)

print(f"{'model':^10} | {'intial':^8} | {'bn':^8} | {'ds':^8} | {'dm':^8} | {'dsbn':^8} | {'dmbn':^8}")
for i in range(20):
    intial_avg = statistics.fmean(intial[i])
    print(f"{labels[i].replace('-dn-bn0',''): <10} | {intial_avg:.3f}    | {statistics.fmean(bn[i])-intial_avg:+.3f}   | {statistics.fmean(ds[i])-intial_avg:+.3f}   | {statistics.fmean(dm[i])-intial_avg:+.3f}   | {statistics.fmean(dsbn[i])-intial_avg:+.3f}   | {statistics.fmean(dmbn[i])-intial_avg:+.3f}")

  model    |  intial  |    bn    |    ds    |    dm    |   dsbn   |   dmbn  
cl1-cf16   | 0.828    | -0.028   | -0.025   | -0.023   | -0.029   | -0.032
cl1-cf32   | 0.839    | -0.033   | -0.026   | -0.022   | -0.032   | -0.041
cl1-cf64   | 0.828    | -0.016   | -0.012   | -0.010   | -0.040   | -0.033
cl1-cf128  | 0.839    | -0.035   | -0.013   | -0.022   | -0.039   | -0.034
cl2-cf16   | 0.832    | -0.010   | -0.001   | +0.002   | +0.004   | -0.020
cl2-cf32   | 0.848    | +0.008   | -0.007   | -0.010   | -0.021   | -0.025
cl2-cf64   | 0.843    | -0.012   | -0.017   | -0.003   | -0.039   | -0.039
cl2-cf128  | 0.846    | -0.002   | -0.008   | -0.011   | -0.028   | -0.005
cl3-cf16   | 0.858    | +0.013   | -0.006   | -0.052   | +0.022   | -0.049
cl3-cf32   | 0.867    | +0.015   | -0.014   | -0.037   | -0.012   | -0.025
cl3-cf64   | 0.871    | +0.012   | -0.013   | -0.022   | -0.035   | -0.051
cl3-cf128  | 0.876    | -0.015   | -0.010   | -0.023   | -0.011   | -0.048
cl4-cf16   | 0.874    |